In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
import statsmodels.api as sm
import statsmodels.stats.api as sms
import seaborn as sns
import scipy.stats as scipy
from scipy.stats import shapiro

In [2]:
df = pd.read_csv("/work/Motor_Vehicle_Collisions_-_Crashes.csv")

/tmp/ipykernel_221/2334064192.py:1: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/work/Motor_Vehicle_Collisions_-_Crashes.csv")


In [3]:
df = df.dropna(subset=["VEHICLE_MAKE", 'DRIVER_LICENSE_STATUS','DRIVER_SEX', 'VEHICLE_YEAR'])
df = df[df['VEHICLE_YEAR'] <= 2025]
df = df[df['VEHICLE_OCCUPANTS'] <= 20]

In [4]:
#are Ford vehicles more prone to fatal accidents
df

,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_YEAR,...,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED
2,16937807,3405925,3/22/2016,5:40,161407,NY,NaN,FORD -CAR/SUV,NaN,1997.0,...,Center Front End,NaN,NaN,NaN,N,NaN,Outside Car Distraction,Driver Inattention/Distraction,0,0
4,16946723,3406521,3/24/2016,15:30,38027,CT,NaN,BMW -CAR/SUV,NaN,2013.0,...,Left Front Bumper,NaN,NaN,NaN,N,NaN,Unsafe Speed,Following Too Closely,0,0
5,16946721,3406521,3/24/2016,15:30,38026,NY,NaN,HYUN -CAR/SUV,NaN,2013.0,...,Right Rear Bumper,Center Front End,NaN,NaN,Unspecified,NaN,Unspecified,Unspecified,0,0
6,16947454,3406526,3/25/2016,12:20,50680,NY,Taxi,FORD -CAR/SUV,FORD ZZZ,2011.0,...,Right Front Quarter Panel,NaN,NaN,NaN,N,NaN,Passing Too Closely,NaN,0,0
7,16972534,3406722,4/15/2016,21:40,156014,NY,NaN,CHEV -CAR/SUV,NaN,2014.0,...,Right Front Quarter Panel,Right Front Bumper,NaN,NaN,N,NaN,Unsafe Lane Changing,Unspecified,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89092,17311768,3518719,9/11/2016,20:30,778066,NY,Station Wagon/Sport Utility Vehicle,FORD -CAR/SUV,NaN,2016.0,...,Center Front End,Left Side Doors,No Damage,No Damage,N,NaN,Traffic Control Disregarded,Failure to Yield Right-of-Way,11,0
89093,17414543,3578653,12/6/2016,3:00,1189706,NJ,Sedan,JEEP -CAR/SUV,NaN,2013.0,...,Center Back End,Right Rear Bumper,Left Rear Bumper,Undercarriage,N,NaN,Alcohol Involvement,Unsafe Speed,11,0
89094,17414544,3578653,12/6/2016,3:00,1189705,NJ,Sedan,HOND -CAR/SUV,NaN,2008.0,...,Center Back End,Left Rear Quarter Panel,NaN,NaN,N,NaN,Alcohol Involvement,Unsafe Speed,11,0
89095,17414542,3578653,12/6/2016,3:00,1189702,NY,Sedan,MERZ -CAR/SUV,NaN,2016.0,...,Center Front End,Left Front Bumper,Right Front Quarter Panel,Right Front Bumper,N,NaN,Alcohol Involvement,Unsafe Speed,11,0


In [5]:
df = df[['CRASH_DATE', 'CRASH_TIME', 'VEHICLE_MAKE', 'VEHICLE_YEAR','VEHICLE_OCCUPANTS', 'DRIVER_SEX', 'DRIVER_LICENSE_STATUS', 
        'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED']].copy()

In [6]:
df = df.drop(df[df['NUMBER OF PERSONS INJURED'] > df['VEHICLE_OCCUPANTS']].index)

In [7]:
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0
...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0


In [8]:
df['isFord'] = df['VEHICLE_MAKE'].apply(lambda x: 1 if isinstance(x, str) and 'FORD' in x.upper() else 0)
df['Fatal'] = df['NUMBER OF PERSONS KILLED'].apply(lambda x: 1 if x > 0 else 0)
df['LicenseStatus'] = df['DRIVER_LICENSE_STATUS'].apply(lambda x: 1 if 'Licensed' in x else 0)

In [9]:
df['Fatal'].value_counts()

Fatal
0    62517
1       36
Name: count, dtype: int64

In [10]:
df['NUMBER OF PERSONS KILLED'].value_counts()

NUMBER OF PERSONS KILLED
0    62517
1       35
3        1
Name: count, dtype: int64

In [11]:
df['DRIVER_LICENSE_STATUS'].value_counts()

DRIVER_LICENSE_STATUS
Licensed      61411
Unlicensed      820
Permit          322
Name: count, dtype: int64

In [12]:
df['VEHICLE_MAKE'].value_counts().head()

VEHICLE_MAKE
TOYT -CAR/SUV    11652
HOND -CAR/SUV     7134
NISS -CAR/SUV     6382
FORD -CAR/SUV     5725
CHEV -CAR/SUV     3146
Name: count, dtype: int64

In [13]:
df.value_counts(df['VEHICLE_MAKE'].str.contains('FORD') & df['Fatal'] == 1)

False    62545
True         8
Name: count, dtype: int64

In [14]:
df.value_counts(df['NUMBER OF PERSONS INJURED'] > 0)

NUMBER OF PERSONS INJURED
False    52547
True     10006
Name: count, dtype: int64

In [15]:
df.value_counts(df['NUMBER OF PERSONS INJURED'])

NUMBER OF PERSONS INJURED
0     52547
1      8617
2      1055
3       226
4        68
5        27
6         8
7         2
9         2
10        1
Name: count, dtype: int64

In [16]:
df.value_counts(df['VEHICLE_OCCUPANTS'] > 0)

VEHICLE_OCCUPANTS
True     59855
False     2698
Name: count, dtype: int64

In [17]:
df['VEHICLE_MAKE'].value_counts().to_csv('vehicle_counts.csv')

In [18]:
df[df['Fatal'] == 1]['VEHICLE_MAKE'].value_counts()

VEHICLE_MAKE
FORD -CAR/SUV      5
TOYT -CAR/SUV      5
CHEV -CAR/SUV      3
FORD               2
MERZ -CAR/SUV      2
GMC -CAR/SUV       2
HOND -CAR/SUV      2
SUZI -MCL          1
BMW -CAR/SUV       1
YAMA -MCL          1
MAZD -CAR/SUV      1
FORD-TRUCK/BUS     1
INTL-TRUCK/BUS     1
HYUN -CAR/SUV      1
ACUR -CAR/SUV      1
AUDI -CAR/SUV      1
VOLV -TRUCK/BUS    1
MERC -CAR/SUV      1
MACK-TRUCK/BUS     1
NE/FL              1
HOND -MCL          1
CHRY -CAR/SUV      1
Name: count, dtype: int64

In [19]:
df['Injured'] = df['NUMBER OF PERSONS INJURED'].apply(lambda x: 1 if x > 0 else 0)

In [20]:
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,Fatal,LicenseStatus,Injured
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,0,1,0
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,0,1,0
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,0,1,0
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,0,1,0
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,0,1,1
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,0,1,1
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,0,1,1
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,0,1,1


In [21]:
logit_y = df['Fatal']

In [22]:
#logit model
x = df['isFord']
x = sm.add_constant(x)
model1 = sm.Logit(logit_y,x).fit()

Optimization terminated successfully.
         Current function value: 0.004835
         Iterations 12


In [23]:
results_model1 = model1.summary()
results_model1

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Fatal   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62551
Method:                           MLE   Df Model:                            1
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                0.006954
Time:                        17:35:01   Log-Likelihood:                -302.44
converged:                       True   LL-Null:                       -304.56
Covariance Type:            nonrobust   LLR p-value:                   0.03958
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.6013      0.189    -40.213      0.000      -7.972      -7.231
isFord         0.9025      0.401      2.250      0.024       0.116       1.689
==============================================================================
"""

In [24]:
df['date'] = pd.to_datetime(df['CRASH_DATE'])
df['time'] = pd.to_datetime(df['CRASH_TIME'])
df['month'] = df['date'].dt.strftime('%m').astype(int)
df['day'] = df['date'].dt.strftime("%A")
df['time'] = df['time'].dt.strftime('%H%M').astype(int)
df

/tmp/ipykernel_221/3027926625.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['CRASH_TIME'])


,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,Fatal,LicenseStatus,Injured,date,time,month,day
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,0,1,0,2016-03-22,540,3,Tuesday
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,0,1,0,2016-03-24,1530,3,Thursday
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,0,1,0,2016-03-24,1530,3,Thursday
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,0,1,0,2016-03-25,1220,3,Friday
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,0,1,0,2016-04-15,2140,4,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,0,1,1,2016-12-01,2104,12,Thursday
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,0,1,1,2018-05-11,2115,5,Friday
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,0,1,1,2016-10-06,855,10,Thursday
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,0,1,1,2016-10-11,0,10,Tuesday


In [25]:
df['peakTime'] = df['time'].apply(lambda x: 1 if x >= 700 and x <= 900 else(1 if x >= 1600 and x <= 1900 else 0))
df['weekday'] = df['day'].apply(lambda x: 1 if x != 'Saturday' or x != 'Sunday' else 0)
df['springSummer'] = df['month'].apply(lambda x: 1 if x <= 8 else 0)
df['isCar'] = df['VEHICLE_MAKE'].apply(lambda x: 1 if isinstance(x, str) and 'CAR'in x.upper() else 0)
df['isBus'] = df['VEHICLE_MAKE'].apply(lambda x: 1 if isinstance(x, str) and 'BUS' in x.upper() else 0)

In [26]:
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,...,Injured,date,time,month,day,peakTime,weekday,springSummer,isCar,isBus
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,...,0,2016-03-22,540,3,Tuesday,0,1,1,1,0
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,0,2016-03-24,1530,3,Thursday,0,1,1,1,0
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,0,2016-03-24,1530,3,Thursday,0,1,1,1,0
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,...,0,2016-03-25,1220,3,Friday,0,1,1,1,0
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,...,0,2016-04-15,2140,4,Friday,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,...,1,2016-12-01,2104,12,Thursday,0,1,0,0,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,...,1,2018-05-11,2115,5,Friday,0,1,1,1,0
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,...,1,2016-10-06,855,10,Thursday,1,1,0,1,0
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,...,1,2016-10-11,0,10,Tuesday,0,1,0,1,0


In [27]:
df['isHonda'] = df['VEHICLE_MAKE'].apply(lambda x : 1 if isinstance(x, str) and 'HON' in x.upper() else 0)
df['isToyota'] = df['VEHICLE_MAKE'].apply(lambda x : 1 if isinstance(x, str) and 'TOY' in x.upper() else 0)

In [28]:
df['driverSex'] = df['DRIVER_SEX'].apply(lambda x: 1 if x == 'M' else 0)
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,...,month,day,peakTime,weekday,springSummer,isCar,isBus,isHonda,isToyota,driverSex
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,...,3,Tuesday,0,1,1,1,0,0,0,1
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,3,Thursday,0,1,1,1,0,0,0,1
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,3,Thursday,0,1,1,1,0,0,0,1
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,...,3,Friday,0,1,1,1,0,0,0,1
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,...,4,Friday,0,1,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,...,12,Thursday,0,1,0,0,0,0,0,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,...,5,Friday,0,1,1,1,0,0,0,1
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,...,10,Thursday,1,1,0,1,0,0,0,1
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,...,10,Tuesday,0,1,0,1,0,0,0,0


In [29]:
x2 = df[['isFord', 'peakTime', 'weekday', 'VEHICLE_YEAR', 'LicenseStatus', 'driverSex', 'VEHICLE_OCCUPANTS', 
        'springSummer', 'isCar', 'isBus']]
x2 = sm.add_constant(x2)

In [30]:
#complex logit model
model3 = sm.Logit(logit_y, x2).fit()
results_model3 = model3.summary()
results_model3

Optimization terminated successfully.
         Current function value: 0.004598
         Iterations 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Fatal   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62543
Method:                           MLE   Df Model:                            9
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                 0.05565
Time:                        17:35:06   Log-Likelihood:                -287.61
converged:                       True   LL-Null:                       -304.56
Covariance Type:            nonrobust   LLR p-value:                 9.298e-05
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.9081      0.406      2.239      0.025       0.113       1.703
peakTime             -1.3686      0.531     -2.578      0.010      -2.409      -0.328
weekday               3.7976     58.545      0.065      0.948    -110.948     118.544
VEHICLE_YEAR         -0.0039      0.029     -0.132      0.895      -0.061       0.053
LicenseStatus        -1.8602      0.538     -3.455      0.001      -2.915      -0.805
driverSex            -0.2272      0.398     -0.571      0.568      -1.007       0.552
VEHICLE_OCCUPANTS     0.0145      0.145      0.100      0.920      -0.270       0.299
springSummer          0.5613      0.483      1.162      0.245      -0.385       1.508
isCar                -1.7622      0.463     -3.807      0.000      -2.669      -0.855
isBus                -0.8766      0.652     -1.344      0.179      -2.155       0.402
=====================================================================================
"""

In [31]:
x3 = df[['isFord', 'isToyota', 'isHonda', 'peakTime', 'weekday', 'VEHICLE_YEAR', 'LicenseStatus', 
            'driverSex', 'VEHICLE_OCCUPANTS', 'springSummer', 'isCar', 'isBus']]
x3 = sm.add_constant(x3)


In [32]:
#logit model with other makes
model5 = sm.Logit(logit_y, x3).fit()
model5_results = model5.summary()
model5_results

Optimization terminated successfully.
         Current function value: 0.004597
         Iterations 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Fatal   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62541
Method:                           MLE   Df Model:                           11
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                 0.05574
Time:                        17:35:07   Log-Likelihood:                -287.58
converged:                       True   LL-Null:                       -304.56
Covariance Type:            nonrobust   LLR p-value:                 0.0003681
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.9306      0.424      2.193      0.028       0.099       1.762
isToyota              0.1213      0.523      0.232      0.816      -0.903       1.146
isHonda               0.0154      0.632      0.024      0.981      -1.224       1.254
peakTime             -1.3675      0.531     -2.576      0.010      -2.408      -0.327
weekday               4.7623     58.669      0.081      0.935    -110.227     119.752
VEHICLE_YEAR         -0.0043      0.029     -0.148      0.882      -0.062       0.053
LicenseStatus        -1.8642      0.539     -3.460      0.001      -2.920      -0.808
driverSex            -0.2331      0.399     -0.585      0.559      -1.015       0.548
VEHICLE_OCCUPANTS     0.0148      0.145      0.102      0.919      -0.270       0.300
springSummer          0.5618      0.483      1.164      0.245      -0.384       1.508
isCar                -1.7872      0.480     -3.726      0.000      -2.727      -0.847
isBus                -0.8797      0.653     -1.348      0.178      -2.159       0.399
=====================================================================================
"""

In [33]:
#negative binomial model
y = df['NUMBER OF PERSONS KILLED']
nb_model = sm.GLM(y, x3, family = sm.families.NegativeBinomial()).fit()
nb_results = nb_model.summary()
nb_results

/root/venv/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


<class 'statsmodels.iolib.summary.Summary'>
"""
                    Generalized Linear Model Regression Results                     
====================================================================================
Dep. Variable:     NUMBER OF PERSONS KILLED   No. Observations:                62553
Model:                                  GLM   Df Residuals:                    62541
Model Family:              NegativeBinomial   Df Model:                           11
Link Function:                          Log   Scale:                          1.0000
Method:                                IRLS   Log-Likelihood:                -300.52
Date:                      Tue, 02 Dec 2025   Deviance:                       499.50
Time:                              17:35:08   Pearson chi2:                 6.90e+04
No. Iterations:                          10   Pseudo R-squ. (CS):          0.0006050
Covariance Type:                  nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                1.1265      0.394      2.862      0.004       0.355       1.898
isToyota              0.0959      0.520      0.184      0.854      -0.924       1.116
isHonda              -0.0098      0.631     -0.016      0.988      -1.246       1.227
peakTime             -1.4332      0.529     -2.708      0.007      -2.470      -0.396
weekday              22.5779     55.445      0.407      0.684     -86.091     131.247
VEHICLE_YEAR         -0.0134      0.028     -0.485      0.628      -0.068       0.041
LicenseStatus        -1.7498      0.540     -3.243      0.001      -2.807      -0.692
driverSex            -0.1317      0.393     -0.335      0.738      -0.903       0.639
VEHICLE_OCCUPANTS     0.1182      0.101      1.169      0.242      -0.080       0.316
springSummer          0.4800      0.482      0.996      0.319      -0.464       1.424
isCar                -1.6693      0.476     -3.506      0.000      -2.602      -0.736
isBus                -0.9278      0.653     -1.420      0.156      -2.208       0.353
=====================================================================================
"""

In [34]:
logit_injured_y = df['Injured']
linear_injured_y = df['NUMBER OF PERSONS INJURED']

In [35]:
#injured logit model
injured_model = sm.Logit(logit_injured_y, x).fit()
injured_model_summary = injured_model.summary()
injured_model_summary

Optimization terminated successfully.
         Current function value: 0.439602
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Injured   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62551
Method:                           MLE   Df Model:                            1
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:               3.783e-06
Time:                        17:35:09   Log-Likelihood:                -27498.
converged:                       True   LL-Null:                       -27499.
Covariance Type:            nonrobust   LLR p-value:                    0.6483
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6568      0.012   -143.874      0.000      -1.679      -1.634
isFord        -0.0164      0.036     -0.455      0.649      -0.087       0.054
==============================================================================
"""

No statistical significance that ford cars cause more fatal accidents. fatal accidents are rare cannot be truly determined with OLS and Logit model

In [36]:
#injured logit model with other makes
injured_model2 = sm.Logit(logit_injured_y, x3).fit()
injured_model_summary2 = injured_model2.summary()
injured_model_summary2

Optimization terminated successfully.
         Current function value: 0.431267
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Injured   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62541
Method:                           MLE   Df Model:                           11
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                 0.01897
Time:                        17:35:10   Log-Likelihood:                -26977.
converged:                       True   LL-Null:                       -27499.
Covariance Type:            nonrobust   LLR p-value:                1.028e-216
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord               -0.0047      0.038     -0.126      0.900      -0.078       0.069
isToyota              0.0779      0.029      2.655      0.008       0.020       0.135
isHonda               0.0085      0.035      0.240      0.811      -0.061       0.078
peakTime             -0.0565      0.024     -2.392      0.017      -0.103      -0.010
weekday              31.4311      3.915      8.028      0.000      23.758      39.104
VEHICLE_YEAR         -0.0164      0.002     -8.396      0.000      -0.020      -0.013
LicenseStatus        -0.4623      0.071     -6.552      0.000      -0.601      -0.324
driverSex            -0.0315      0.025     -1.264      0.206      -0.080       0.017
VEHICLE_OCCUPANTS     0.2625      0.010     27.136      0.000       0.244       0.281
springSummer          0.1032      0.038      2.707      0.007       0.028       0.178
isCar                -0.0529      0.056     -0.941      0.347      -0.163       0.057
isBus                -0.8434      0.082    -10.344      0.000      -1.003      -0.684
=====================================================================================
"""

In [37]:
#linear injured model with other makes
linear_injured_model2 = sm.OLS(linear_injured_y, x3).fit()
linear_injured_summary2 = linear_injured_model2.summary()
linear_injured_summary2

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     NUMBER OF PERSONS INJURED   R-squared:                       0.049
Model:                                   OLS   Adj. R-squared:                  0.049
Method:                        Least Squares   F-statistic:                     291.7
Date:                       Tue, 02 Dec 2025   Prob (F-statistic):               0.00
Time:                               17:35:10   Log-Likelihood:                -42568.
No. Observations:                      62553   AIC:                         8.516e+04
Df Residuals:                          62541   BIC:                         8.527e+04
Df Model:                                 11                                         
Covariance Type:                   nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.0051      0.006      0.786      0.432      -0.008       0.018
isToyota              0.0144      0.005      2.772      0.006       0.004       0.025
isHonda               0.0034      0.006      0.547      0.585      -0.009       0.016
peakTime             -0.0145      0.004     -3.564      0.000      -0.023      -0.007
weekday               5.1243      0.697      7.350      0.000       3.758       6.491
VEHICLE_YEAR         -0.0025      0.000     -7.168      0.000      -0.003      -0.002
LicenseStatus        -0.0720      0.014     -5.028      0.000      -0.100      -0.044
driverSex            -0.0056      0.004     -1.286      0.198      -0.014       0.003
VEHICLE_OCCUPANTS     0.1053      0.002     54.195      0.000       0.101       0.109
springSummer          0.0144      0.007      2.108      0.035       0.001       0.028
isCar                 0.0064      0.010      0.640      0.522      -0.013       0.026
isBus                -0.0955      0.012     -7.649      0.000      -0.120      -0.071
==============================================================================
Omnibus:                    43334.546   Durbin-Watson:                   0.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           884073.753
Skew:                           3.098   Prob(JB):                         0.00
Kurtosis:                      20.344   Cond. No.                     7.33e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [38]:
#negative binomial model for injured
nb2_model = sm.GLM(linear_injured_y, x3, family = sm.families.NegativeBinomial()).fit()
nb2_results = nb2_model.summary()
nb2_results

/root/venv/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     NUMBER OF PERSONS INJURED   No. Observations:                62553
Model:                                   GLM   Df Residuals:                    62541
Model Family:               NegativeBinomial   Df Model:                           11
Link Function:                           Log   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -31536.
Date:                       Tue, 02 Dec 2025   Deviance:                       33571.
Time:                               17:35:12   Pearson chi2:                 5.23e+04
No. Iterations:                           15   Pseudo R-squ. (CS):            0.03707
Covariance Type:                   nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.0143      0.034      0.417      0.677      -0.053       0.082
isToyota              0.0702      0.027      2.577      0.010       0.017       0.124
isHonda               0.0161      0.033      0.495      0.620      -0.048       0.080
peakTime             -0.0737      0.022     -3.362      0.001      -0.117      -0.031
weekday              25.4256      3.615      7.033      0.000      18.340      32.511
VEHICLE_YEAR         -0.0136      0.002     -7.552      0.000      -0.017      -0.010
LicenseStatus        -0.3363      0.065     -5.208      0.000      -0.463      -0.210
driverSex            -0.0363      0.023     -1.590      0.112      -0.081       0.008
VEHICLE_OCCUPANTS     0.3735      0.007     56.510      0.000       0.361       0.386
springSummer          0.0721      0.035      2.050      0.040       0.003       0.141
isCar                 0.0507      0.053      0.960      0.337      -0.053       0.154
isBus                -0.7382      0.076     -9.720      0.000      -0.887      -0.589
=====================================================================================
"""

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f2ce982c-ab7e-4df7-957e-2768a79f0182' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>